In [15]:
from helper import combine

_code = '''{{ command }} {{ head }}:
    {{ body }}'''


_for = combine({
    'main': _code,
    'command': 'for',
    'head': '{{ pointer }} in {{ collection }}',
})

_if = combine({
    'main': _code,
    'command': 'if',
    'head': '{{ condition }}',
})

_else = combine({
    'main': _code,
    'command': 'else',
    'head': '',
})


_def = combine({
    'main': _code,
    'command': 'def',
    'head': '{{ name }}({{ parameter }})'
})

_call = '{{ name }}({{ parameter }})'


invoke = combine({
    'main': _call,
    'parameter': ''
})

_var = '{{ name }} = {{ value }}\n'

_write = combine({
    'main': _call,
    'name':'print',
    'parameter':'{{ text }}'
})



def _import(imports):
    result = ''
    for i in imports:
        result += combine(
        {
            'main': 'from {{ module }} import {{ component }}\n' if type(i) == list else 'import {{ module }}\n',
            'module': i[0] if type(i) == list else i,
            'component': i[1] if type(i) == list else ''
        })
    return result

function = combine({
    'main'         : _def,
    'name'         : 'bar',
    'parameter'    : 'array',
    'body'         : _for,
    'collection'   : 'array',
    'pointer'      : 'i',
    'body 2'       : _for,
    'collection 2' : 'i',
    'pointer 2'    : 'j',
    'body 3'       : _write,
    'text'         : 'j'
})

print(function)

def bar(array):
    for i in array:
        for j in i:
            print(j)


In [13]:
print(_import([
                'tensorflow as tf',
                ['tensorflow', 'keras'],
                'cv2',
                'numpy as np',
                'matplotlib.pyplot as plt',
            ]))

def parameter(input):
    result = ''
    for i in input.keys():
        result += combine({
            'main': '{{ name }} = {{ value }},',
            'name': i,
            'value': input[i]
        })+', '
    if(len(result) > 0):
        result = result[:-2]
    return result

def compile(learing_rate = None, decay = None):
    optimizer_parameters = {}
    if learing_rate == None and decay == None:
        optimizer = '\'adam\''
    else:
        optimizer = 'keras.optimizers.Adam({{ parameters }})'
        if(learing_rate != None):
            optimizer_parameters['lr'] = learing_rate
        if(decay != None):
            optimizer_parameters['decay'] = decay
    return combine({
        'main': '''{{ model }}.compile({{ parameters }})''',
        'parameters': parameter({
            'optimizer': optimizer,
            'loss': '\'{{ loss }}\'',
            'metrics': '[\'accuracy\']'
        ,}),
        'parameters 2': parameter(optimizer_parameters)})

print(compile(learing_rate='0.001', decay='1e-6'))
train = '''{{ model }}.fit({{ x }}, {{ y }}, epochs={{ epoch }}, batch_size={{ batch }})'''

import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np
import matplotlib.pyplot as plt

{{ model }}.compile(optimizer = keras.optimizers.Adam(lr = 0.001,, decay = 1e-6,),, loss = '{{ loss }}',, metrics = ['accuracy'],)
